In [1]:
import pandas as pd
import numpy as np
import os 
import glob
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, jaccard_score

# Generate RANDOM Baseline

In [2]:
# load answer sheet
answer_sheet = pd.read_csv('../data/fingergesture/answer_sheet.csv', index_col=0)
answer_sheet = answer_sheet.values.flatten()

In [3]:
# generate random array filled with 1 or zero

recall_list = []
precision_list = []
f1_list = []
accuracy_list = []
jaccard_list = []

for i in range(5000):
    random_array = np.random.randint(2, size=answer_sheet.shape)
    recall = recall_score(answer_sheet, random_array)
    precision = precision_score(answer_sheet, random_array)
    f1 = f1_score(answer_sheet, random_array)
    accuracy = accuracy_score(answer_sheet, random_array)
    jaccard = jaccard_score(answer_sheet, random_array)

    recall_list.append(recall)
    precision_list.append(precision)
    f1_list.append(f1)
    accuracy_list.append(accuracy)
    jaccard_list.append(jaccard)
    
print(f"recall: {np.mean(recall_list):.3f} ± {np.std(recall_list):.3f}")
print(f"precision: {np.mean(precision_list):.3f} ± {np.std(precision_list):.3f}")
print(f"f1: {np.mean(f1_list):.3f} ± {np.std(f1_list):.3f}")
print(f"accuracy: {np.mean(accuracy_list):.3f} ± {np.std(accuracy_list):.3f}")
print(f"jaccard: {np.mean(jaccard_list):.3f} ± {np.std(jaccard_list):.3f}")


recall: 0.500 ± 0.070
precision: 0.501 ± 0.050
f1: 0.499 ± 0.056
accuracy: 0.501 ± 0.050
jaccard: 0.334 ± 0.050


# Get QR Score Results (Mine)

In [10]:
max_columns = ['Recall_max', 'Precision_max', 'F1_max',
       'Accuracy_max', 'Jaccard_max', 'Flag',
       'Model_name', 'exp', 'model_accuracy']

In [11]:
qr_results = pd.read_csv("dataset/handgesture/qr_handgesture_cwri_results.csv")
qr_results['qr_lambda'] = 0.0

In [12]:
# 8100, 8200, 8300, 8400 -> 0.1
# 8101, 8201, 8301, 8401 -> 0.2
# 8102, 8202, 8302, 8402 -> 0.5
# 8103, 8203, 8303, 8403 -> 1.0

def set_qr_lambda(df, list_of_exp_name, qr_lambda):
    df.loc[df['EXP_NAME'].isin(list_of_exp_name), 'qr_lambda'] = qr_lambda
    return df

qr_results = set_qr_lambda(qr_results, [8100, 8200, 8300, 8400], 0.1)
qr_results = set_qr_lambda(qr_results, [8101, 8201, 8301, 8401], 0.2)
qr_results = set_qr_lambda(qr_results, [8102, 8202, 8302, 8402], 0.5)
qr_results = set_qr_lambda(qr_results, [8103, 8203, 8303, 8403], 1.0)

In [13]:
without_qr = qr_results[qr_results['EXP_NAME'].isin([8000, 8001, 8002, 8003])]
# get mean and std, sort Model_name by shufflenet, resnet9, mlpmixer, simplevit
without_qr.groupby('Model_name')[max_columns].agg(['mean', 'std'])

/tmp/ipykernel_1510735/510591188.py:3: FutureWarning: ['Flag', 'Model_name', 'exp'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  without_qr.groupby('Model_name')[max_columns].agg(['mean', 'std'])


Recall_max           Precision_max              F1_max            \
                 mean       std          mean       std      mean       std   
Model_name                                                                    
mlpmixer        0.400  0.083666      0.671287  0.071768  0.496483  0.079294   
resnet9         0.316  0.243064      0.588819  0.123331  0.357380  0.198480   
shufflenet      0.356  0.055498      0.551870  0.037335  0.428901  0.032759   
simplevit       0.508  0.079498      0.539180  0.022295  0.520160  0.042943   

           Accuracy_max           Jaccard_max           model_accuracy  \
                   mean       std        mean       std           mean   
Model_name                                                               
mlpmixer          0.600  0.046368    0.333111  0.068718       0.719914   
resnet9           0.530  0.027386    0.231980  0.149434       0.684607   
shufflenet        0.530  0.015811    0.273435  0.026436       0.652207   
simplevit         0.536  0.018166    0.352397  0.038783       0.493165   

                      
                 std  
Model_name            
mlpmixer    0.021783  
resnet9     0.044159  
shufflenet  0.139100  
simplevit   0.029836

In [14]:
qr_results.groupby(['Model_name','qr_lambda'])[max_columns].agg(['mean', 'std'])

/tmp/ipykernel_1510735/2698150697.py:1: FutureWarning: ['Flag', 'Model_name', 'exp'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  qr_results.groupby(['Model_name','qr_lambda'])[max_columns].agg(['mean', 'std'])


Recall_max           Precision_max              F1_max  \
                           mean       std          mean       std      mean   
Model_name qr_lambda                                                          
mlpmixer   0.0            0.400  0.083666      0.671287  0.071768  0.496483   
           0.1            0.768  0.118828      0.534843  0.020940  0.627033   
           0.2            0.904  0.071274      0.536536  0.049741  0.671463   
           0.5            0.840  0.143527      0.623829  0.101892  0.700379   
           1.0            0.832  0.155949      0.637752  0.083976  0.708783   
resnet9    0.0            0.316  0.243064      0.588819  0.123331  0.357380   
           0.1            0.616  0.107145      0.618276  0.062190  0.613487   
           0.2            0.732  0.108259      0.661083  0.066301  0.691567   
           0.5            0.728  0.067231      0.648992  0.089130  0.684342   
           1.0            0.796  0.130690      0.681937  0.084473  0.724278   
shufflenet 0.0            0.356  0.055498      0.551870  0.037335  0.428901   
           0.1            0.528  0.246414      0.610829  0.065726  0.527386   
           0.2            0.584  0.162727      0.640914  0.107989  0.603896   
           0.5            0.668  0.146014      0.684814  0.076833  0.668871   
           1.0            0.832  0.152709      0.591909  0.078605  0.678856   
simplevit  0.0            0.508  0.079498      0.539180  0.022295  0.520160   
           0.1            0.436  0.101390      0.580528  0.051402  0.491905   
           0.2            0.412  0.109179      0.536191  0.022499  0.459618   
           0.5            0.412  0.179778      0.600746  0.086792  0.462623   
           1.0            0.496  0.122801      0.592640  0.041734  0.531186   

                               Accuracy_max           Jaccard_max            \
                           std         mean       std        mean       std   
Model_name qr_lambda                                                          
mlpmixer   0.0        0.079294        0.600  0.046368    0.333111  0.068718   
           0.1        0.035459        0.548  0.022804    0.457481  0.037849   
           0.2        0.041118        0.556  0.069857    0.506598  0.047784   
           0.5        0.018577        0.636  0.093434    0.539160  0.021918   
           1.0        0.053301        0.660  0.078740    0.551038  0.063963   
resnet9    0.0        0.198480        0.530  0.027386    0.231980  0.149434   
           0.1        0.069623        0.616  0.059414    0.445389  0.072769   
           0.2        0.069746        0.676  0.066933    0.532148  0.084509   
           0.5        0.068980        0.662  0.081363    0.523679  0.084139   
           1.0        0.051524        0.698  0.069065    0.569758  0.062482   
shufflenet 0.0        0.032759        0.530  0.015811    0.273435  0.026436   
           0.1        0.201229        0.586  0.071274    0.375737  0.161343   
           0.2        0.128573        0.628  0.098590    0.442242  0.131678   
           0.5        0.090127        0.676  0.080187    0.508445  0.110142   
           1.0        0.022724        0.606  0.079875    0.514202  0.026327   
simplevit  0.0        0.042943        0.536  0.018166    0.352397  0.038783   
           0.1        0.068728        0.558  0.037014    0.328413  0.061344   
           0.2        0.075007        0.528  0.021679    0.300838  0.063136   
           0.5        0.121548        0.554  0.049800    0.307272  0.100628   
           1.0        0.055141        0.572  0.022804    0.363211  0.052226   

                     model_accuracy            
                               mean       std  
Model_name qr_lambda                           
mlpmixer   0.0             0.719914  0.021783  
           0.1             0.720883  0.019447  
           0.2             0.715985  0.017445  
           0.5             0.715501  0.017069  
           1.0             0.713940  0.015512  
r

In [8]:
with_qr = qr_results[qr_results['qr_lambda'] == 1.0]
with_qr.groupby('Model_name')[max_columns].agg(['mean', 'std'])

/tmp/ipykernel_1510735/728128271.py:2: FutureWarning: ['Flag', 'Model_name', 'exp'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  with_qr.groupby('Model_name')[max_columns].agg(['mean', 'std'])


Recall_max           Precision_max              F1_max            \
                 mean       std          mean       std      mean       std   
Model_name                                                                    
mlpmixer        0.832  0.155949      0.637752  0.083976  0.708783  0.053301   
resnet9         0.796  0.130690      0.681937  0.084473  0.724278  0.051524   
shufflenet      0.832  0.152709      0.591909  0.078605  0.678856  0.022724   
simplevit       0.496  0.122801      0.592640  0.041734  0.531186  0.055141   

           Accuracy_max           Jaccard_max           EXP_NAME      CV_NUM  \
                   mean       std        mean       std     mean  std   mean   
Model_name                                                                     
mlpmixer          0.660  0.078740    0.551038  0.063963   8303.0  0.0    2.0   
resnet9           0.698  0.069065    0.569758  0.062482   8203.0  0.0    2.0   
shufflenet        0.606  0.079875    0.514202  0.026327   8103.0  0.0    2.0   
simplevit         0.572  0.022804    0.363211  0.052226   8403.0  0.0    2.0   

                     model_accuracy            
                 std           mean       std  
Model_name                                     
mlpmixer    1.581139       0.713940  0.015512  
resnet9     1.581139       0.676372  0.007637  
shufflenet  1.581139       0.444510  0.206221  
simplevit   1.581139       0.462271  0.028626